In [2]:
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import urllib.request
import urllib.parse
import json
import random

In [2]:
tree = ET.parse('data/volnamista/vm20150723.xml')
root = tree.getroot()

In [3]:
out = {}

In [4]:
for misto in root:
    uid = misto.attrib['uid']
    celkemVm = misto.attrib['celkemVm']
    out[uid] = {'celkemVm': celkemVm}
    for val in misto:
        tag = str(val.tag).split('}')[1]
        for attr in val.attrib:
            out[uid][tag + '_' + attr] =  val.attrib[attr]

In [5]:
outFrame = pd.DataFrame.from_dict(out, orient='index')

In [6]:
outFrame.columns

Index(['MZDA_min', 'MODRE_KARTY_pocetVmProMk', 'PRACPRAVNI_VZTAH_kod',
       'UVAZEK_kod', 'ZELENE_KARTY_vmRezervProVyhovZk', 'URAD_PRACE_nazev',
       'PRACOVISTE_okresKod', 'MZDA_max', 'ZAMEST_KARTY_vmRezervProVyhovZm',
       'PRACOVISTE_psc', 'MODRE_KARTY_vmRezervProPodanMk',
       'ZELENE_KARTY_celkemVmProZk', 'ZAMEST_KARTY_vmRezervProZm',
       'ZAMEST_KARTY_celkemVmProZm', 'MIN_VZDELANI_nazev',
       'ZELENE_KARTY_vmRezervProPodanZk', 'VHODNE_PRO_absolventyVs',
       'VHODNE_PRO_ozp', 'MODRE_KARTY_vmRezervProVydanMk', 'PRACOVISTE_cobce',
       'MIN_VZDELANI_kod', 'VHODNE_PRO_bezbar', 'VHODNE_PRO_absolventySs',
       'PROFESE_nazev', 'ZAMEST_KARTY_pocetVmProZm',
       'MODRE_KARTY_celkemVmProMk', 'PRACOVISTE_okres', 'OBOR_nazev',
       'URAD_PRACE_kod', 'PROFESE_doplnek', 'MODRE_KARTY_vmRezervProVyhovMk',
       'ZAMEST_KARTY_vmRezervProVydanZm', 'MODRE_KARTY_vmRezervProMk',
       'PROFESE_kod', 'ZELENE_KARTY_typZk', 'ZELENE_KARTY_klicovyPersonal',
       'ZELENE_KARTY

In [9]:
outFrame.OBOR_nazev.value_counts()

Výroba a provoz            14147
Obchod a cestovní ruch      8987
Služby                      3945
Doprava                     2876
Stavebnictví                2541
Zdravotnictví               2055
Obrana a ochrana            1677
Výchova a vzdělávání        1301
Finance                     1243
Administrativa              1097
Informační technologie       912
Zemědělství a lesnictví      857
Management                   209
Kultura a sport              165
Právo                         76
Věda a výzkum                 48
dtype: int64

In [15]:
def addrjoin(ul, cp, ob):
    if (ul == ul):
        return ul + ' ' + str(cp) + ', '+ ob
    else:
        return str(cp) + ', ' + ob

In [17]:
outFrame['addr'] = outFrame.apply(lambda row: addrjoin(row['PRACOVISTE_ulice'], row['PRACOVISTE_cp'], row['PRACOVISTE_obec']), axis=1)

In [22]:
def gcode(address):
    params = urllib.parse.urlencode({'SearchText': address, 'ExtraInformation': 'standard'})
    url = "http://localhost/ruian/rest.py/Geocode/json?&%s" % params
    with urllib.request.urlopen(url) as f:
        jsn = json.loads(f.read().decode('utf-8'))
        if (len(jsn['records']) == 0):
            return np.nan, np.nan
        else:
            coords = jsn['records'][random.randint(0, len(jsn['records'])-1)]
            return (float(coords['JTSKY']) * -1, float(coords['JTSKX']) * -1)

In [23]:
buffer = {}

In [24]:
unmatched = pd.read_csv(r'scratch/unmatched.txt')

In [30]:
unmatched['x'] = np.nan
unmatched['y'] = np.nan

In [40]:
#geocoder
for i, row in unmatched.iterrows():
    if (row[3] == row[3]):
        continue
    else:
        if ((i % 100) == 0):
            print(i)
        if row[2] not in buffer:
            crds = gcode(row[2].replace('nan, ', ''))
            unmatched.ix[i, 3] = crds[0]
            unmatched.ix[i, 4] = crds[1]

            buffer[row[2]] = {'X': crds[0],'Y': crds[1]}
        else:
            unmatched.ix[i, 3] = buffer[row[2]]['X']
            unmatched.ix[i, 4] = buffer[row[2]]['Y']

300
700
1200
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100


In [20]:
outFrame.to_csv(r'data/adresy2.csv', encoding='utf-8')

In [48]:
unmatched

,OID,Field1,addr,x,y
0,-1,1.198421e+09,"118, Všeruby",-857227.25,-1110310.63
1,-1,1.334408e+08,"245, Vinařice",-765369.81,-1031042.99
2,-1,1.344691e+09,"236, Všeruby",-831706.38,-1057788.50
3,-1,1.368151e+09,"135, Zahořany",-855236.92,-1100182.84
4,-1,1.381508e+08,"100, Rozvadov",-878804.56,-1068553.88
5,-1,1.395307e+08,"102, Studenec",-656704.00,-1001593.00
6,-1,1.411708e+08,"102, Studenec",-656704.00,-1001593.00
7,-1,1.509407e+08,"118, Blatno",-640311.36,-1092762.19
8,-1,1.524531e+09,"397, Podlesí",NaN,NaN
9,-1,1.558011e+09,"125, Litomyšl",-611514.82,-1082878.34


In [47]:
unmatched.to_csv(r'data/unmatched_aftermatch.csv', encoding='utf-8')

In [3]:
#importer
dat = pd.read_csv(r'data/unmatched_aftermatch.csv')

In [4]:
dat.to_excel(r'data/unmatched_aftermatch.xlsx', index=False)